In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
df = pd.read_csv('../data/yoochoose-clicks.dat', header=None)
df.columns=['session_id', 'timestamp', 'item_id', 'category']
buy_df = pd.read_csv('../data/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id','timestamp' , 'item_id' , 'price' , 'quantity']
item_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id)
df.head()


d:\project\vscode\GNN\GCN_Proj\thevenv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,2053,0
1,1,2014-04-07T10:54:09.868Z,2052,0
2,1,2014-04-07T10:54:46.998Z,2054,0
3,1,2014-04-07T10:57:00.306Z,9876,0
4,2,2014-04-07T13:56:37.614Z,19448,0


In [3]:
# 采样数据
import numpy as np
sampled_session_id = np.random.choice(df.session_id.unique(), 100000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    100000
timestamp     357479
item_id        20372
category         119
dtype: int64

In [4]:
# 标签
df['labels'] = df.session_id.isin(buy_df.session_id)
df.head()
# session_id:用户id	timestamp：时间戳	item_id：商品id	category：商品种类	labels：是否购买

,session_id,timestamp,item_id,category,labels
1373,456,2014-04-01T16:47:29.980Z,28987,0,False
1374,456,2014-04-01T16:49:28.482Z,29011,0,False
1375,456,2014-04-01T16:52:24.010Z,29021,0,False
1726,571,2014-04-03T14:14:52.663Z,41261,0,False
1727,571,2014-04-03T14:15:18.856Z,39396,0,False


* 咱们把每一个session_id都当作一个图,每一个图具有多个点和一个标签
* 其中每个图中的点就是其item_id，特征咱们暂且用其id来表示，之后会做embedding
### 数据集制作流程
* 1.首先遍历数据中每一组session_id，目的是将其制作成(from torch_geometric.data import Data)格式
* 2.对每一组session_id中的所有item_id进行编码(例如15453,3651,15452)就按照数值大小编码成(2,0,1)
* 3.这样编码的目的是制作edge_index，因为在edge_index中我们需要从0，1，2，3...开始
* 4.点的特征就由其ID组成，edge_index是这样，因为咱们浏览的过程中是有顺序的比如(0,0.2,1)
* 5.所以边就是0->0,0->2,2->1这样的，对应的索引就为target_nodes: [0 ,2,1], source_nodes: [0,0,2]
* 6.最后转换格式data = Data(x=x, edge_index=edge_index,y=y)
* 7.最后将数据集保存下来(以后就不用重复处理了)


In [11]:
# 制作数据集
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
import torch
from torch_geometric.data import Data

class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root, transform, pre_transform) # transform就是数据增强，对每一个数据都执行
        self.data, self.slices = torch.load(self.processed_paths[0])
 
    @property #python装饰器， 只读属性，方法可以像属性一样访问
    def raw_file_names(self): #①检查self.raw_dir目录下是否存在raw_file_names()属性方法返回的每个文件 
                              #②如有文件不存在，则调用download()方法执行原始文件下载
        return []
    @property
    def processed_file_names(self): #③检查self.processed_dir目录下是否存在self.processed_file_names属性方法返回的所有文件，有则直接加载
                                    #④没有就会走process,得到'yoochoose_click_binary_1M_sess.dataset'文件
        return ['yoochoose_click_binary_1M_sess.dataset']
 
    def download(self):#①检查self.raw_dir目录下是否存在raw_file_names()属性方法返回的每个文件 
                       #②如有文件不存在，则调用download()方法执行原始文件下载
        pass
    
    def process(self):#④没有就会走process,得到'yoochoose_click_binary_1M_sess.dataset'文件
        
        data_list = [] #保存最终生成图的结果
 
        # process by session_id
        grouped = df.groupby('session_id')
        for session_id, group in tqdm(grouped):
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values
 
            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]
 
            edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
            x = node_features
 
            y = torch.FloatTensor([group.labels.values[0]])
            #创建图
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)#转换成可以保存到本地的格式
        torch.save((data, slices), self.processed_paths[0])#保存操作，名字跟yoochoose_click_binary_1M_sess.dataset一致

In [12]:
dataset = YooChooseBinaryDataset(root='../data/')

Processing...
100%|██████████| 100000/100000 [01:48<00:00, 922.46it/s]
Done!


In [7]:
# 单独拿出process过程做一下解释
from torch_geometric.data import InMemoryDataset   #数据格式
from tqdm import tqdm   #进度条
df_test = df[:100]      #取前100个
grouped = df_test.groupby('session_id')  #基于session_id分组
for session_id, group in tqdm(grouped):  #遍历每一组的session_id，都做成一个图
    print('session_id:',session_id)
    #LabelEncoder：sklearn中的包,对数值做转换
    sess_item_id = LabelEncoder().fit_transform(group.item_id)#把item_id做一个转换，转换成从0开始的格式，赋值给sess_item_id
    print('sess_item_id:',sess_item_id)
    group = group.reset_index(drop=True)#重置索引
    group['sess_item_id'] = sess_item_id
    print('group:',group)
    #设置点的标签为item_id    drop_duplicates:去除重复项的操作
    node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values
    print('node_features:',node_features)
    node_features = torch.LongTensor(node_features).unsqueeze(1)  #unsqueeze:指定的位置插入一个维度
    print('node_features:',node_features)
    print('node_features:',node_features.shape) # torch.Size([3, 1])
 
    #因为是顺序结构，所以邻接矩阵可以通过这种方式构建
    target_nodes = group.sess_item_id.values[1:]#取出target
    source_nodes = group.sess_item_id.values[:-1]#取出source
    print('target_nodes:',target_nodes)
    print('source_nodes:',source_nodes)
    # 指定边索引
    edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
    x = node_features
    y = torch.FloatTensor([group.labels.values[0]])
    print(f"y:{y}")
    data = Data(x=x, edge_index=edge_index, y=y)
    print('data:',data)

  0%|          | 0/19 [00:00<?, ?it/s]d:\project\vscode\GNN\GCN_Proj\thevenv\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
d:\project\vscode\GNN\GCN_Proj\thevenv\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
d:\project\vscode\GNN\GCN_Proj\thevenv\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
d:\project\vscode\GNN\GCN_Proj\thevenv\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
d:\project\vscode\GNN\GCN_Proj\thevenv\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
d:\project\vscode\GNN\GCN_P

session_id: 456
sess_item_id: [0 1 2]
group:    session_id                 timestamp  item_id category  labels  \
0         456  2014-04-01T16:47:29.980Z    28987        0   False   
1         456  2014-04-01T16:49:28.482Z    29011        0   False   
2         456  2014-04-01T16:52:24.010Z    29021        0   False   

   sess_item_id  
0             0  
1             1  
2             2  
node_features: [28987 29011 29021]
node_features: tensor([[28987],
        [29011],
        [29021]])
node_features: torch.Size([3, 1])
target_nodes: [1 2]
source_nodes: [0 1]
y:tensor([0.])
data: Data(x=[3, 1], edge_index=[2, 2], y=[1])
session_id: 571
sess_item_id: [1 0]
group:    session_id                 timestamp  item_id category  labels  \
0         571  2014-04-03T14:14:52.663Z    41261        0   False   
1         571  2014-04-03T14:15:18.856Z    39396        0   False   

   sess_item_id  
0             1  
1             0  
node_features: [39396 41261]
node_features: tensor([[39396],
  

### 构建网络模型
模型可以任选，这里只是举例而已   
跟图像中的卷积和池化操作非常类似，最后在全连接输出   
网络模型比较简单，我就不做解释了，直接上代码。   
其中TopKPooling类似于下采样，是剪枝的过程，选择得分比较低的节点剪枝掉，然后再重新组合成一个新的图。

#### topkpooling
![](../img/topkpooling.jpg)

In [8]:
embed_dim = 128
from torch_geometric.nn import TopKPooling,SAGEConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module): #针对图进行分类任务
    def __init__(self):
        super(Net, self).__init__()
 
        self.conv1 = SAGEConv(embed_dim, 128) #卷积层 输入embed_dim，输出128
        self.pool1 = TopKPooling(128, ratio=0.8) #做剪枝操作
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +10, embedding_dim=embed_dim)#映射向量
        self.lin1 = torch.nn.Linear(128, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch # x:n*1,其中每个图里点的个数是不同的
        #print(x)
        x = self.item_embedding(x)# n*1*128 特征编码后的结果
        #print('item_embedding',x.shape)
        x = x.squeeze(1) # n*128        
        #print('squeeze',x.shape)
        
        """
        对输入不断做卷积，不断做池化池化，得到的特征会越来越浓缩，图会越来越小，
        但是池化完成之后的特征维度都是一样的
        
        """
        x = F.relu(self.conv1(x, edge_index))# n*128
        #print('conv1',x.shape)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)# pool之后得到 n*0.8个点
        #print('self.pool1',x.shape)
        #print('self.pool1',edge_index)
        #print('self.pool1',batch)
        #x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
        x1 = gap(x, batch)  #   gap:全局平均池化  得到全局特征
        #print('gmp',gmp(x, batch).shape) # batch*128
        #print('cat',x1.shape) # batch*256
        x = F.relu(self.conv2(x, edge_index))
        #print('conv2',x.shape)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        #print('pool2',x.shape)
        #print('pool2',edge_index)
        #print('pool2',batch)
        #x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
        x2 = gap(x, batch)
        #print('x2',x2.shape)
        x = F.relu(self.conv3(x, edge_index))
        #print('conv3',x.shape)
        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        #print('pool3',x.shape)
        #x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
        x3 = gap(x, batch)
        #print('x3',x3.shape)# batch * 256
        x = x1 + x2 + x3 # 获取不同尺度的全局特征
        """通过全连接层，得到最终输出结果值"""
        x = self.lin1(x)
        #print('lin1',x.shape)
        x = self.act1(x)
        x = self.lin2(x)
        #print('lin2',x.shape)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)
 
        x = torch.sigmoid(self.lin3(x)).squeeze(1)#batch个结果
        #print('sigmoid',x.shape)
        return x

In [13]:
from torch_geometric.loader import DataLoader
 
def train():
    model.train()
 
    loss_all = 0
    for data in train_loader:#遍历dataloader
        data = data
        #print('data',data)
        optimizer.zero_grad()
        output = model(data)#data数据传入模型
        label = data.y
        loss = crit(output, label)#计算损失
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()#梯度更新
    return loss_all / len(dataset)
    
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
crit = torch.nn.BCELoss()
train_loader = DataLoader(dataset, batch_size=64)
for epoch in range(10):
    print('epoch:',epoch)
    loss = train()
    print(loss)
 
from  sklearn.metrics import roc_auc_score
 
def evalute(loader,model):
    model.eval()
 
    prediction = []
    labels = []
 
    with torch.no_grad():
        for data in loader:
            data = data#.to(device)
            pred = model(data)#.detach().cpu().numpy()
 
            label = data.y#.detach().cpu().numpy()
            prediction.append(pred)
            labels.append(label)
    prediction =  np.hstack(prediction)
    labels = np.hstack(labels)
 
    return roc_auc_score(labels,prediction) 
 
 
for epoch in range(1):
    roc_auc_score = evalute(dataset,model)
    print('roc_auc_score',roc_auc_score)

epoch: 0
0.2146196381545067
epoch: 1
0.19208341326892375
epoch: 2
0.17321355732738972
epoch: 3
0.15442424919128417
epoch: 4
0.1366953404724598
epoch: 5
0.12191478551328182
epoch: 6
0.10955878899037838
epoch: 7
0.09894091757774354
epoch: 8
0.09006955304473639
epoch: 9
0.08333975611388683
roc_auc_score 0.9282249362593582
